In [5]:
vec1 = ['large', 'light double bed', 'light pouf', 'white bedside table', 'light round chandelier', 'white painting', '2 wall lamps']
vec2 = ['small', 'black bed', 'painting', 'wall lamp', 'green curtains']
vec3 = ['small', 'white double bed', 'white dresser', 'white mirror', 'light table lamp', 'small black TV', 'white ceiling fan', 'white blinds']
vec4 = ['large', 'light wood-colored double bed', '2 light wood-colored bedside tables', '2 light wood-colored dressers', 'light wood-colored wardrobe', 'white rug', 'black chair', 'chandelier', 'light yellow curtains']
vecs = [vec1, vec2, vec3, vec4]

In [11]:
import numpy as np
# !pip install spacy
# !pip install gensim
import spacy
import gensim.downloader as api

In [12]:
word2vec_model = api.load("word2vec-google-news-300")
nlp = spacy.load("en_core_web_sm")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [19]:
# Function to calculate cosine similarity between two vectors
def calculate_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Function to calculate element similarity using Jaccard similarity
def calculate_element_similarity(set1, set2):
    return len(set1 & set2) / len(set1 | set2)

# Function to preprocess and get vector representations for vectors
def get_vector_representation(vector, word2vec_model):
    vec = []
    for phrase in vector:
        tokens = nlp(phrase)
        for token in tokens:
            word = token.text
            if word in word2vec_model:
                vec.extend(word2vec_model[word])
    return vec

In [16]:
vector1 = get_vector_representation(vec1, word2vec_model)
vector2 = get_vector_representation(vec2, word2vec_model)
vector3 = get_vector_representation(vec3, word2vec_model)
vector4 = get_vector_representation(vec4, word2vec_model)

In [21]:
max_dimension = max(len(vector1), len(vector2), len(vector3), len(vector4))

def pad_vector(vector, dimension):
    return np.pad(vector, (0, dimension - len(vector)), 'constant')

vector1 = pad_vector(vector1, max_dimension)
vector2 = pad_vector(vector2, max_dimension)
vector3 = pad_vector(vector3, max_dimension)
vector4 = pad_vector(vector4, max_dimension)

In [17]:
element_similarities = []
element_similarities.append(calculate_element_similarity(set(vec1), set(vec2)))
element_similarities.append(calculate_element_similarity(set(vec1), set(vec3)))
element_similarities.append(calculate_element_similarity(set(vec1), set(vec4)))
element_similarities.append(calculate_element_similarity(set(vec2), set(vec3)))
element_similarities.append(calculate_element_similarity(set(vec2), set(vec4)))
element_similarities.append(calculate_element_similarity(set(vec3), set(vec4)))

In [22]:
vector_similarities = []
vector_similarities.append(calculate_cosine_similarity(vector1, vector2))
vector_similarities.append(calculate_cosine_similarity(vector1, vector3))
vector_similarities.append(calculate_cosine_similarity(vector1, vector4))
vector_similarities.append(calculate_cosine_similarity(vector2, vector3))
vector_similarities.append(calculate_cosine_similarity(vector2, vector4))
vector_similarities.append(calculate_cosine_similarity(vector3, vector4))

In [23]:
overall_similarities = [0.5 * vector_sim + 0.5 * element_sim for vector_sim, element_sim in zip(vector_similarities, element_similarities)]

In [24]:
for i, similarity in enumerate(overall_similarities):
    print(f"Similarity between vec{1+i//3} and vec{2+i%3}: {similarity:.2f}")

Similarity between vec1 and vec2: 0.07
Similarity between vec1 and vec3: 0.16
Similarity between vec1 and vec4: 0.12
Similarity between vec2 and vec2: 0.15
Similarity between vec2 and vec3: 0.06
Similarity between vec2 and vec4: 0.08
